In [1]:
!pip install polars beautifulsoup4 splinter selenium scikit-learn altair vl-convert-python tensorflow keras-tuner pandas

In [2]:
#Data Manipulation and display toools
import polars as pl
from polars.exceptions import InvalidOperationError
import altair as alt
import numpy as np
import pathlib

from IPython.display import display_html 
#Necessary for tensorflow on my machine due to distutils being depreciated

# Data preprocessing tools
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Machine learning tools
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

#Necessary for tensorflow on my machine due to distutils being depreciated
import setuptools
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
import keras_tuner as kt

random_state = 2112250415

2024-10-13 17:32:38.117584: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-13 17:32:38.118103: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-13 17:32:38.119912: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-13 17:32:38.125160: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-13 17:32:38.133907: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

# Loading Data

In [3]:
cwd = pathlib.Path.cwd()

if cwd.name == 'Mild-Steel-Tempering':
    print("Path is project root")
else:
    print("Please correct current working directory to the project root")


Path is project root


In [4]:
resources_path = pathlib.PurePath(pathlib.PurePath(cwd), 'resources', 'pred_hardness')
resources_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Mild-Steel-Tempering/resources/pred_hardness')

In [5]:
images_path = pathlib.PurePath(pathlib.PurePath(cwd), 'images', 'pred_hardness')
images_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Mild-Steel-Tempering/images/pred_hardness')

# Loading Scraped data from the other file

In [370]:
data = pl.read_csv(f'{resources_path.parent}/all_cols_post_scrape.csv')
data.shape

In [371]:
data.tail(3)

(1286, 20)

# Select columns that have relevant data for the analysis

Exclude:
* Source data - does not provide information about the problem
* initial hardness - very incomplete data and no accurate way to fill it
* searchable -  contains more information than searchable, but is likely relipcated in elemental
  * Searchable contains fewer values, this grouping was used to scrape the strength values and contains no additional information
  * steel_type contains steel denotation from the papers. The informationa in this column is replicated in the elemental composition columns
  * PCA will handle this


In [372]:
df_relevent = data.drop(['source', 'searchable', 'initial_hardness_post_quenching_HRC'])
df_relevent.columns

['steel_type',
 'tempering_time_s',
 'tempering_temperature_C',
 'C',
 'Mn',
 'P',
 'S',
 'Si',
 'Ni',
 'Cr',
 'Mo',
 'V',
 'Al',
 'Cu',
 'final_hardness_post_tempering_HRC',
 'yield_strength_MPa',
 'ultimate_strength_MPA']

# Preprocessing the data for machine learning

In [373]:
target_columns = ['final_hardness_post_tempering_HRC']

X_prepre = df_relevent.drop(target_columns)

y_prepre = df_relevent.select(target_columns)

In [374]:
y_prepre.head(3)

final_hardness_post_tempering_HRC
f64
50.6
48.3
43.7


In [375]:
y_prepre.describe()

statistic,final_hardness_post_tempering_HRC
str,f64
"""count""",1286.0
"""null_count""",0.0
"""mean""",41.810653
"""std""",14.373455
"""min""",0.9
"""25%""",32.5
"""50%""",43.6
"""75%""",52.3
"""max""",68.5


# Preprocess target values

Target values are continuious.

TODO: determine preprocessing, if any. Test effects of scaling vs not.

In [376]:
y_scaler = StandardScaler()

y_scaler.fit(y_prepre)

y = y_scaler.transform(y_prepre)

# Preprocess data values

In [377]:
print(X_prepre.shape)
X_prepre.head(3)

(1286, 16)


steel_type,tempering_time_s,tempering_temperature_C,C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,yield_strength_MPa,ultimate_strength_MPA
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64
"""AISI-SAE 1026""",600,204.4,0.25,0.79,0.012,0.026,0.11,0.0,0.0,0.0,0.0,0.0,0.0,415,490
"""AISI-SAE 1026""",600,260.0,0.25,0.79,0.012,0.026,0.11,0.0,0.0,0.0,0.0,0.0,0.0,415,490
"""AISI-SAE 1026""",600,315.6,0.25,0.79,0.012,0.026,0.11,0.0,0.0,0.0,0.0,0.0,0.0,415,490


# Encode and scale variables
Most columns are numeric and range from 0.0# to 1000 so scaling is necessary
steel names are categorical and would benefit from one-hot encoding.

In [378]:
to_scale = X_prepre.drop('steel_type')
check_onehot = X_prepre.select('steel_type')

## One hot encode steel types

Encoding chosen because this is a categorical for a given material and testing suite

### Unify steel names so there is only 1 column per steel code

In [379]:
print(len(check_onehot.unique()))
# correct duplicate name 
dict_rename.update({'AISI-SAE 1030': "AISI 1030"})
to_onehot = check_onehot['steel_type']\
    .replace(dict_rename)\
    .str.strip_chars()
[print(val) for val in to_onehot.unique()]
print(len(to_onehot.unique()))

30
AISI 6145
AISI-SAE 1035
AISI 1074 Carbon Steel
AISI-SAE 5140
AISI-SAE 4140
AISI 1090
AISI-SAE 1080
AISI-SAE 1038
AISI 52100
AISI-SAE 4027
AISI 1055
AISI-SAE 1040
AISI 4047
AISI 4640
AISI-SAE 5160
AISI-SAE 1050
AISI-SAE 1065
AISI-SAE 6150
AISI-SAE 4340
AISI-SAE 9254
AISI-SAE 1335
AISI-SAE 1042
AISI 1049
AISI-SAE 4037
AISI 1030
AISI 1095
AISI-SAE 1026
AISI-SAE 1045
28


In [380]:
to_onehot = pl.DataFrame(to_onehot)
to_onehot.shape

(1286, 1)

In [381]:
# create the transformers
ohe = OneHotEncoder(handle_unknown='ignore')

oh_transformed = ohe.fit_transform(to_onehot)

# get the transformed column names
ohe_names = ohe.get_feature_names_out()

# 4 stack exchange methods tried. Then I made this one up ¯\_(ツ)_/¯
# Combinated of recenly changed API and polars I think, no way to rename all columns like in pandas.
columns = [f'column_{x}' for x in range(0, 30)]
rename_dict = dict(zip(columns, ohe_names))

df_ohe_encoded = pl.DataFrame(oh_transformed.toarray()).rename(rename_dict)
df_ohe_encoded.tail(3)

steel_type_AISI 1030,steel_type_AISI 1049,steel_type_AISI 1055,steel_type_AISI 1074 Carbon Steel,steel_type_AISI 1090,steel_type_AISI 1095,steel_type_AISI 4047,steel_type_AISI 4640,steel_type_AISI 52100,steel_type_AISI 6145,steel_type_AISI-SAE 1026,steel_type_AISI-SAE 1035,steel_type_AISI-SAE 1038,steel_type_AISI-SAE 1040,steel_type_AISI-SAE 1042,steel_type_AISI-SAE 1045,steel_type_AISI-SAE 1050,steel_type_AISI-SAE 1065,steel_type_AISI-SAE 1080,steel_type_AISI-SAE 1335,steel_type_AISI-SAE 4027,steel_type_AISI-SAE 4037,steel_type_AISI-SAE 4140,steel_type_AISI-SAE 4340,steel_type_AISI-SAE 5140,steel_type_AISI-SAE 5160,steel_type_AISI-SAE 6150,steel_type_AISI-SAE 9254
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Scale numerical columns 
This will ensure the small elemental composition values are not overshadowed by the large strength properties 

In [384]:
to_scale.tail(3)

tempering_time_s,tempering_temperature_C,C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,yield_strength_MPa,ultimate_strength_MPA
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64
86400,500.0,1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,525,685
86400,600.0,1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,525,685
86400,700.0,1.15,0.58,0.012,0.021,0.09,0.0,0.01,0.0,0.0,0.0,0.0,525,685


In [385]:
scaler = StandardScaler()

X_scaler = scaler.fit(to_scale)

scaled = scaler.transform(to_scale)

scaler_names = scaler.get_feature_names_out()
columns = [f'column_{x}' for x in range(0, 14)]
rename_dict = dict(zip(columns, scaler_names))

df_scaled = pl.DataFrame(scaled).rename(rename_dict)
df_scaled.tail(3)


tempering_time_s,tempering_temperature_C,C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,yield_strength_MPa,column_14
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.89775,0.467987,2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,0.188703,-0.105424
1.89775,1.028689,2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,0.188703,-0.105424
1.89775,1.589391,2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,0.188703,-0.105424


## Merge one hot encoded variables with scaled numeric variables

In [386]:
X = pl.concat([df_ohe_encoded, df_scaled], how='horizontal')
X

steel_type_AISI 1030,steel_type_AISI 1049,steel_type_AISI 1055,steel_type_AISI 1074 Carbon Steel,steel_type_AISI 1090,steel_type_AISI 1095,steel_type_AISI 4047,steel_type_AISI 4640,steel_type_AISI 52100,steel_type_AISI 6145,steel_type_AISI-SAE 1026,steel_type_AISI-SAE 1035,steel_type_AISI-SAE 1038,steel_type_AISI-SAE 1040,steel_type_AISI-SAE 1042,steel_type_AISI-SAE 1045,steel_type_AISI-SAE 1050,steel_type_AISI-SAE 1065,steel_type_AISI-SAE 1080,steel_type_AISI-SAE 1335,steel_type_AISI-SAE 4027,steel_type_AISI-SAE 4037,steel_type_AISI-SAE 4140,steel_type_AISI-SAE 4340,steel_type_AISI-SAE 5140,steel_type_AISI-SAE 5160,steel_type_AISI-SAE 6150,steel_type_AISI-SAE 9254,tempering_time_s,tempering_temperature_C,C,Mn,P,S,Si,Ni,Cr,Mo,V,Al,Cu,yield_strength_MPa,column_14
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.617861,-1.189448,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,-0.342255,-1.009596
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.617861,-0.877697,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,-0.342255,-1.009596
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.617861,-0.565947,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,-0.342255,-1.009596
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.617861,-0.254758,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,-0.342255,-1.009596
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.617861,0.056993,-1.155898,0.167872,-0.60703,0.268364,-0.513467,-0.392117,-0.813526,-0.638219,-0.201129,0.0,-0.337311,-0.342255,-1.009596
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.89775,-0.653417,2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,0.188703,-0.105424
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.89775,-0.092715,2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,0.188703,-0.105424
0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.89775,0.467987,2.69532,-0.618793,-0.60703,-0.394811,-0.591857,-0.392117,-0.791743,-0.638219,-0.201129,0.0,-0.337311,0.188703,-0.105424


In [387]:
X.write_csv(f"{resources_path}/X_nopreproc.csv")

pl.DataFrame(y).write_csv(f"{resources_path}/y_nopreproc.csv")


## Modelling for data preprocess

PCA modelling used to reduce X dimensions from 44 to 12 maintaining 95.7% of the explained variance.

KBinsDiscretizer used to bin imbalanced Y target. 15 bins, where half have less than 5 support reduced to 5 similarly supported bins

### PCA analysis

Introduction of two strength metrics certainly introduced multcolinearity, with eachother and with the elemental composision, some of whicha re indicators of strength.

PCA analysis will be done to alleiviate these factors.

In [388]:
X = pl.read_csv(f"{resources_path}/X_nopreproc.csv")

In [389]:
pca_explained_sum = {'x': [], 'pca_explained_var': []}
pca_explained_min = {'x': [], 'pca_explained_min_component': []}
for x in range(3,30):
    pca = PCA(n_components=x ,random_state=random_state)

    pca.fit(X)

    pca_explained_sum['x'].append(x)
    sum_explained = np.sum(pca.explained_variance_ratio_)
    pca_explained_sum['pca_explained_var'].append(sum_explained)

    pca_explained_min['x'].append(x)
    min_explained = np.min(pca.explained_variance_ratio_)
    pca_explained_min['pca_explained_min_component'].append(min_explained)

In [390]:
df_pca_explained = pl.DataFrame(pca_explained_sum)

### PCA component number determination

In [391]:
alt.Chart(df_pca_explained,
          title= "Total Explained Variance vs Number of PCA Features").mark_line().encode(
    alt.X('x:Q').title("PCA X"),
    alt.Y('pca_explained_var:Q')\
        .scale(domain=(0.5,1))\
            .title("Total explained variance")
)

alt.Chart(...)

### Value of increasing PCA components decreases between 10 and 15

In [392]:
df_pca_explained_min = pl.DataFrame(pca_explained_min)
df_pca_explained_min.filter(pl.col('x').le(15) & pl.col('x').ge(10))

x,pca_explained_min_component
i64,f64
10,0.037895
11,0.025717
12,0.015481
13,0.01162
14,0.006842
15,0.003192


### PCA component number 13 selected
Component 13 has half the explained ratio contribution as component 13, and component 14 decreased by a factor of 10

13 components has explains 96.2% of the variance.

In [393]:
df_pca_explained.filter(pl.col("x").eq(13))

x,pca_explained_var
i64,f64
13,0.961645


In [394]:
pca = PCA(n_components= 13, random_state=random_state)

pca.fit(X)

X_pca = pca.transform(X)

df_pca = pl.DataFrame(X_pca)
df_pca.head(3)

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-1.396549,0.357322,0.11312,-0.35176,-0.475294,-0.085418,-1.457699,0.792878,-0.123649,-0.967397,-0.402453,0.138054,0.316879
-1.408114,0.303587,0.136813,-0.349753,-0.573852,-0.23484,-1.270245,0.633357,-0.123784,-0.940428,-0.405776,0.141606,0.304695
-1.419679,0.249852,0.160507,-0.347746,-0.67241,-0.384263,-1.082791,0.473836,-0.123919,-0.913459,-0.409099,0.145159,0.292511


In [395]:
df_pca.describe()

statistic,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0,1286.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",-3.8677e-17,1.9338e-17,-8.9785e-17,8.5641e-17,3.3151e-17,2.7626e-18,3.1770e-17,5.5252e-18,-4.4202e-17,-4.2820e-17,-1.2432e-16,1.6576e-17,-1.6576e-17
"""std""",1.653448,1.572193,1.326167,1.210351,1.076369,1.003431,0.975282,0.911914,0.811145,0.753117,0.620412,0.481368,0.417029
"""min""",-2.755651,-2.749125,-2.631238,-2.526038,-3.325919,-1.580944,-2.390954,-2.977647,-1.634804,-1.47535,-1.006469,-0.940166,-0.97989
"""25%""",-1.107186,-1.236004,-0.652254,-0.596507,-0.673487,-0.753635,-0.706409,-0.567779,-0.509937,-0.442693,-0.411362,-0.270313,-0.360951
"""50%""",-0.188674,-0.121004,-0.113746,-0.187207,-0.016828,-0.195364,-0.037295,-0.023571,0.066648,-0.027211,-0.168751,0.012771,0.094007
"""75%""",0.473879,1.091142,0.54757,0.852972,0.593213,0.34491,0.575031,0.474041,0.529772,0.249033,0.422331,0.249639,0.301865
"""max""",4.895881,3.739467,4.450363,3.646042,4.063784,3.125609,4.122047,3.998631,1.510213,2.563479,1.49625,1.012016,0.760051


## Check data balance before train-test-split
Balancing the target is unnecessary as it is a continuous variable. 

It is not effected by imbalance in the same way a categorical target is.

In [396]:
df_pca.write_csv(f'{resources_path}/df_X.csv')
pl.DataFrame(y).write_csv(f'{resources_path}/df_y.csv')


# Modelling

In [397]:
X = pl.read_csv(f'{resources_path}/df_X.csv')
y = pl.read_csv(f'{resources_path}/df_y.csv')
# Polars does not always work well with Keras
X = X.to_numpy()
y = y.to_numpy()

## Test train split

In [398]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=random_state)

## Gradient boost regression

The final hardness is a continuos varaible, that may be non-linear with respect to the supporting features. 

### This model does perform well.

I will attmpt to use neural network modelling to see if it can be improved, and to get history.


In [399]:
gb_regressor = GradientBoostingRegressor(random_state=random_state)

gb_regressor.fit(X_train, y_train)


/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Mild-Steel-Tempering/.venv/lib/python3.12/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


GradientBoostingRegressor(random_state=2112250415)

In [400]:
y_pred = gb_regressor.predict(X_test)
y_pred.shape

(322,)

In [401]:
# match y_test shape to model output shape
def reshape_y_for_eval(y_test):
    return y_test.reshape(1,-1)[0]

In [402]:
print("R2 score:", round(gb_regressor.score(X_test, reshape_y_for_eval(y_test)),3))

R2 score: 0.928


### Invert the y_scaling done prior to modelling for the plotting 

In [403]:
def y_inverse_transform(y_data):
    result = y_scaler.inverse_transform(y_data.reshape(-1, 1)).reshape(1,-1)[0]
    return result

In [404]:
y_test_invscal = y_inverse_transform(y_test)
y_pred_invscal = y_inverse_transform(y_pred)

In [405]:
test_df = pl.DataFrame({'Real Final Hardness (HRC)' : y_test_invscal,
                        'Predicted Final Hardness (HRC)' : y_pred_invscal})
test_df.head(3)

Real Final Hardness (HRC),Predicted Final Hardness (HRC)
f64,f64
57.5,56.934392
60.4,60.633832
47.5,45.07313


In [407]:
base = alt.Chart(test_df).mark_point().encode(
    alt.X('Real Final Hardness (HRC)'),
    alt.Y('Predicted Final Hardness (HRC)')
)

fit = base.transform_regression(
    'Real Final Hardness (HRC)','Predicted Final Hardness (HRC)', method='linear'
).mark_line(color='red').encode()

plot = (base + fit).properties(title='GBRegression Fit')
plot.save(f"{images_path}/GBR_fit.png")
plot

alt.LayerChart(...)

## Gradient boost regression as a good fit

R2 score is 0.92 without scaling. This is quite good.

R2 score is 0.93 with y-scaling. Marginally better.

Moving to a Deep Neural Network model to try and improve performance. 

# Deep Neural Network Model

In [408]:
dim_input = X_test.shape[1]
dim_output = y.shape[1]
print("Xdim", dim_input,"| ydim", dim_output)

Xdim 13 | ydim 1


In [409]:
# https://machinelearningmastery.com/multi-label-classification-with-deep-learning/

# input
layers = [Dense(16, activation='relu'),
          # fork model to predict hardness
          Dense(units=16, activation='relu'),
        #Dense(units=64, activation='relu'),
        #Dense(units=64, activation='relu'),
          # Output layers for each label
          Dense(dim_output, name='Hardness')
]

nn_model = Sequential(layers)

nn_model.compile(optimizer=tf.keras.optimizers.Adam(0.0005),
                 loss='mean_absolute_error',
                 metrics=['mse'])

nn_history = nn_model.fit(x=X_train, y=y_train, 
                        verbose=0, 
                        epochs=100,
                        validation_split = 0.2)


In [410]:
nn_model.summary()

Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_54 (Dense)                │ (None, 16)             │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hardness (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,541 (6.02 KB)

 Trainable params: 513 (2.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,028 (4.02 KB)

## Evaluate NN model

Fit and model done 5 times to get stability from the accuracies generated per model.

In [411]:
result = nn_model.evaluate(X_test, y_test, return_dict=True)
result

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 451us/step - loss: 0.1901 - mse: 0.0718


{'loss': 0.18134644627571106, 'mse': 0.06577352434396744}

In [412]:
history_df = pl.DataFrame(nn_history.history)
history_df = history_df.with_row_index()
history_df.head(3)

index,loss,mse,val_loss,val_mse
u32,f64,f64,f64,f64
0,0.913528,1.264116,0.849921,1.064733
1,0.820101,1.024939,0.789308,0.937838
2,0.754411,0.886089,0.742033,0.847306


In [413]:
def plot_model_history(df, name='DNN', feature='mse', tuner=False):
    title = f'{feature}'.title() if feature != 'mse' else "Mean Squared Error"
    selector = ['index', feature]
    if tuner: selector.append(f'val_{feature}')

    df_reduced = df.select(selector)

    plot = alt.Chart(df_reduced).mark_line().encode(
        alt.X('index').title('Epochs'),
        alt.Y(alt.repeat('layer'), type='quantitative').\
            title(title),
        color = alt.datum(alt.repeat('layer'))
    ).repeat(
        layer = selector[1:]
    ).properties(title=f"{title} by Epoch")

    plot.show()

    plot.save(f"{images_path}/{name}_{feature}.png")


In [414]:
plot_model_history(history_df, feature='mse', name="initial_NN", tuner=True)
plot_model_history(history_df, feature='loss', name="initial_NN", tuner=True)

alt.RepeatChart(...)

alt.RepeatChart(...)

# Results

NN model pushes R2 down to 0.5 with very few epochs (<25).

# Build a Tuner

In [464]:
name = "hardness_DNN_tuner"
def create_hp_model(hp):
    #Activation for all hidden layers
      activation = hp.Choice('activation',['relu','tanh','sigmoid'])

      layers = [Dense(units=hp.Int('time_units1',
                             min_value=8,
                             max_value=256,
                             sampling='linear'), 
                             activation=activation),
                             # fork model to predict hardness
                  Dense(units=hp.Int('time_units2',
                             min_value=8,
                             max_value=256,
                             sampling='linear'), 
                             activation=activation),
                  Dense(units=hp.Int('time_units3',
                             min_value=8,
                             max_value=256,
                             sampling='linear'), 
                             activation=activation),
                  Dense(units=hp.Int('time_units4',
                             min_value=8,
                             max_value=256,
                             sampling='linear'), 
                             activation=activation),
                  Dense(dim_output, name='Hardness')
          ]
    
      nn_model = Sequential(layers)

      nn_model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                 loss='mean_absolute_error',
                 metrics=['mse'])
    
      return nn_model

In [465]:
tuner = kt.Hyperband(
    create_hp_model,
    objective=["val_loss","val_loss"],
    max_epochs=20,
    factor=5,
    hyperband_iterations=2,
    project_name=name,
    executions_per_trial=5,
    overwrite= True)# TODO: remove when done tuning this set and have saved DNN to github after model freeze


In [ ]:
tuner.search(x=X_train, 
             y=y_train,
             epochs=250,
             validation_data=(X_test,y_test))

Trial 2 Complete [00h 00m 49s]
multi_objective: 0.2552858471870422

Best multi_objective So Far: 0.19914618730545045
Total elapsed time: 00h 01m 38s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
tanh              |relu              |activation
104               |131               |time_units1
14                |271               |time_units2
445               |398               |time_units3
441               |449               |time_units4
4                 |4                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
1                 |1                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/4
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.4966 - mse: 0.4232 - val_loss: 0.3276 - val_mse: 0.1802
Epoch 2/4
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3469 - mse: 0.2013 - val_loss: 0.2744 - val_mse: 0.1223
Epoch 3/4
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2705 - mse: 0.1213 - 

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7bf04bcfd280>>
Traceback (most recent call last):
  File "/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Mild-Steel-Tempering/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Epoch 1/4
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.4233 - mse: 0.3124 - val_loss: 0.3255 - val_mse: 0.1695
Epoch 2/4
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3072 - mse: 0.1692 - val_loss: 0.2877 - val_mse: 0.1487
Epoch 3/4
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2988 - mse: 0.1536 - val_loss: 0.2754 - val_mse: 0.1300
Epoch 4/4
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2655 - mse: 0.1243 - val_loss: 0.2789 - val_mse: 0.1436


In [453]:
best_model = tuner.get_best_models(1)[0]

In [454]:
fit_history = best_model.fit(x=X_train, 
               y=y_train, 
               verbose=0,
               epochs=150)

In [455]:
tuner.get_best_hyperparameters(1)[0].values

{'activation': 'relu',
 'time_units1': 145,
 'time_units2': 389,
 'time_units3': 70,
 'time_units4': 36,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0016'}

In [456]:
history_df = pl.DataFrame(fit_history.history)
history_df = history_df.with_row_index()
history_df.head(3)

index,loss,mse
u32,f64,f64
0,0.182897,0.062156
1,0.147749,0.045775
2,0.142241,0.042619


In [457]:
plot_model_history(history_df, feature='mse', name=name, tuner=False)
plot_model_history(history_df, feature='loss', name=name, tuner=False)

alt.RepeatChart(...)

alt.RepeatChart(...)

# Optimizing optimizer

In [458]:

def create_opt_model(opt):
    #Activation for all hidden layers

    layers = [Dense(units=164, 
                    activation='relu'),
                Dense(units=24, 
                      activation='relu'),
                Dense(dim_output, name='Hardness')
          ]
    
    nn_model = Sequential(layers)

    nn_model.compile(optimizer=opt,
                 loss='mean_absolute_error',
                 metrics=['mse'])
    
    return nn_model

In [459]:
optimizers = [name.lower() for name in ["Adadelta", 
                                        "Adafactor", 
                                        "Adagrad", 
                                        "Adam", 
                                        "AdamW", 
                                        "Adamax", 
                                        "Ftrl", 
                                        "Lion", 
                                        "Nadam", 
                                        "RMSprop", 
                                        "SGD"]]

In [460]:
for opt in optimizers:
    best_opt = 'starter'
    best_mse = 4

    model = create_opt_model(opt)
    model.fit(x=X_train, 
            y=y_train,
            verbose=0,
            epochs=150)
    
    loss, mse = nn_model.evaluate(X_test, y_test)
    
    if mse < best_mse:
        best_mse = mse
        best_opt = opt

print("Best Optimizer:", best_opt, "with MSE", best_mse)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1901 - mse: 0.0718  
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 580us/step - loss: 0.1901 - mse: 0.0718
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 664us/step - loss: 0.1901 - mse: 0.0718
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 534us/step - loss: 0.1901 - mse: 0.0718
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step - loss: 0.1901 - mse: 0.0718
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - loss: 0.1901 - mse: 0.0718
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step - loss: 0.1901 - mse: 0.0718
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - loss: 0.1901 - mse: 0.0718
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 602us/step - loss: 0.1901 - mse: 0.0718
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step - loss: 0.1901 - mse: 0.0718
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - loss: 0.1901 - mse: 0.0718
Best Optimizer: sgd with MSE 0.06577352434396744


## Epoch optimization

In [461]:
model = create_opt_model(tf.keras.optimizers.SGD(0.01))
fit = model.fit(x=X_train, 
         y=y_train, 
         verbose=0,
         epochs=100)



In [462]:
name= 'epoch_test'
history_df = pl.DataFrame(fit.history)
history_df = history_df.with_row_index()
history_df
plot_model_history(history_df, feature='mse', name=name)
plot_model_history(history_df, feature='loss', name=name)

alt.RepeatChart(...)

alt.RepeatChart(...)

# Model Selected, performance is poor

In [463]:
model = create_opt_model(opt)
fit = model.fit(x=X_train, 
        y=y_train,
        verbose=0,
        epochs=25)
name = 'final NN'
history_df = pl.DataFrame(fit.history)
history_df = history_df.with_row_index()
history_df
plot_model_history(history_df, feature='mse', name=name)
plot_model_history(history_df, feature='loss', name=name)

alt.RepeatChart(...)

alt.RepeatChart(...)